采用互信息来作为特征筛选
---
本文主要是采用特征选择里面的包装法来进行特征筛选，通过对cnews中文文本进行处理，可以达到特征筛选的目的

In [4]:
import jieba
import os

In [5]:
# 确定文件的路径
file_path="../data/cnews/cnews.test.txt"

**步骤1**：读取文件，将label和text分开

In [6]:
labels,messages=list(),list()
with open(file_path,'r',encoding='utf-8') as fp:
    texts=fp.readlines()
    for text in texts:
        text=text.split('\t')
        labels.append(text[0])
        messages.append(text[1])
labels[:3]

['体育', '体育', '体育']

**步骤二**：分词，去除停用词

In [13]:
with open('../data/stop_words_uft8.txt','r',encoding='utf-8') as fp:
    stpwords=fp.readlines()
stopwords=list()
for word in stpwords:
    stopwords.append(word.strip())
stopwords[:5]

[',', '?', '、', '。', '“']

In [14]:
# 去除停用词
test_x = [[each0 for each0 in jieba.cut(each) if each0 not in stopwords] for each in messages]
len(test_x)

10000

In [15]:
# test_x有10000个样本，取500:1500之间的1000个样本为例子,同时取出label
test_x1 = test_x[500:1500]


# 因为test_x1存放的是单个词语构成的list，现在将其按照' '将其为字符串形式
test_x1 = [' '.join(each) for each in test_x1] 
test_x1[1]

'去年 痛 绿衫 军竟 忘记   真当 95 火箭 新浪 体育讯 今天 波士顿 凯尔特人 驾临 华府 所有人 翻开 绿衫 军 首发 名单 时 傻眼 巨头 及拉简 隆多均 高挂 免战牌 凯尔特人 为此 付出 代价 苦战 一个 加时 仍以 94 95 负于 华盛顿 奇才 彻底 无缘 东部 第二 昨天 凯尔特人 惨败 本赛季 曾 三次 战胜 热火 对手 反超 一个身 位 热火 未来 对手 分别 亚特兰大 老鹰 多伦多 猛龙 凯尔特人 今天 奇才 战罢 收官战 还要 面对 本赛季 曾 三次 战胜 纽约 尼克斯 热火 交战 成绩 占优 争 第二 本来 毫无希望 君不见 今天 热火 对决 老鹰 巨头 齐齐 亮相 轻视 奇才 未必 本赛季 前 一次 做客 华府 绿衫 军全 主力 出战 依然 大热 倒灶 怎会 天真 以为 此番 副 选 阵容 必胜 绿衫 军 状态 糟糕 足够 底气 资格 轻视 对手 对此 波士顿 环球 报 无奈 OK 所有 想 巨头 球迷 洗洗 睡 道 格 里 弗斯 已 决定 老将 休息 今天 华府 进行 一场 奇怪 比赛 以全 主力 出战 未必 抢 回 第二 人 留下 口实 人 质疑 绿衫 军 18 座 冠军 抱持 何种 心态 1989 年 以来 赛区 第三 低 排名 夺冠 1994 95 赛季 休斯敦 火箭 2001 02 赛季 洛杉矶 湖人 2003 04 赛季 底特律 活塞 2006 07 赛季 马刺 几次 特殊 情况 火箭 当年 开局 不利 赛季 中期 交易 克莱德 德雷克斯勒 牺牲 战绩 促 磨合 季后赛 时 火箭 实力 已远超 排名 西部 第六 总决赛 对手 新兵 蛋子 奥兰多 魔术 2002 年 湖人 位居 西部 第三 马刺 战绩 相同 马刺 赛区 头名 成为 名义 第二 西部 半决赛 湖人 一个 主场 同理 2004 年 新泽西 篮网 能排 第二 赛区 头名 活塞 手握 实际上 主场优势 2007 年 西部 第三 马刺 夺冠 拜 最强 对手 达拉斯 小牛 首轮 遭遇 黑 八所 赐 骑士 总决赛 菜鸟 也就是说 多年 真正 至少 三连客 晋级 方式 夺冠 球队 队中 95 年 火箭 一支 凯尔特人 上赛季 险些 再次 书写 传奇 东部 第四 身份 出战 先后 第二 第三轮 淘汰 排名 高于 骑士 魔术 总决赛 对手 洛杉矶 湖人 1995 年 魔术 

步骤3：文本向量化

In [16]:
# 将文本向量化
from sklearn.feature_extraction.text import TfidfVectorizer
# 初始化后直接feed给模型
transformer=TfidfVectorizer(ngram_range=(1,2),min_df=0.2, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
tf_idf=transformer.fit_transform(test_x1)
tf_idf_matrix=tf_idf.toarray()

In [18]:
# 取出相应的label,将其离散化
labels_x1=labels[500:1500]
char2num=dict(zip(list(set(labels_x1)),range(len(set(labels_x1)))))
char2num

{'体育': 0, '娱乐': 1}

In [21]:
X_labelnum = []
for each in labels_x1:
    X_labelnum.append(char2num[each])


In [23]:
tf_idf_matrix.shape()

(1000, 62)

步骤4：计算样本与特征之间的互信息

In [30]:
# 现计算样本label与样本特征之间一一的互信息
tf_idf_matrix_T = tf_idf_matrix.T
X_MI = {} # 用于存储tf_idf_matrix_T特征与label之间的互信息值
from sklearn import metrics as mr
import numpy as np
X_labels = np.array(X_labelnum)
for i in range(tf_idf_matrix_T.shape[0]):
    X_MI[i] = mr.mutual_info_score(X_labels,tf_idf_matrix_T[i])

In [31]:
X_MI

{0: 0.24619826572194375,
 1: 0.14522977235188136,
 2: 0.14734647651463229,
 3: 0.16740174049361115,
 4: 0.4034030622503526,
 5: 0.1536535282057453,
 6: 0.19570520200756553,
 7: 0.20990843232597797,
 8: 0.19455075708137606,
 9: 0.16633030232276236,
 10: 0.19369112636560065,
 11: 0.16739700871710742,
 12: 0.2551920933429799,
 13: 0.20259951921572728,
 14: 0.5428524740217371,
 15: 0.2731074874796676,
 16: 0.3488441926564956,
 17: 0.2897688674350848,
 18: 0.31808366843365193,
 19: 0.1422161015337544,
 20: 0.29356515320090654,
 21: 0.16263006881211767,
 22: 0.3794654626012728,
 23: 0.31943327605555716,
 24: 0.20706098165646547,
 25: 0.27186005501996524,
 26: 0.2002218140627043,
 27: 0.452993759900236,
 28: 0.21251810559691148,
 29: 0.19204408298638884,
 30: 0.5851166818798166,
 31: 0.5428524740217371,
 32: 0.30016036017557496,
 33: 0.43714680147354773,
 34: 0.25922700287577616,
 35: 0.15959821836588517,
 36: 0.1836438452099457,
 37: 0.14294953917544295,
 38: 0.43928617857707036,
 39: 0.1528

步骤5：特征筛选，选择出互信息最大的20个特征

In [32]:
# 筛选特征，根据互信息的大小进行从大到小排序，留下前20的特征
X_MIF = [each for each in X_MI if X_MI[each]>=sorted(X_MI.values())[10]]
print('选择的特征对应列序号：',X_MIF)
X_fea = [tf_idf_matrix[each] for each in X_MIF]
X_matrix_fea = np.array(X_fea).T

选择的特征对应列序号： [0, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 38, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60]
